# Merging csv features files into one big merged csv file.

This whole notebook merges all the csv files by merging all columns of feature values into one big merged csv file for each person based on its Id that appears in the audio recording name.

In [ ]:
# ========= B1) Unzip both zip files into one folder =========
import zipfile
from pathlib import Path

extract_root = Path("/content/all_feature_csvs")
extract_root.mkdir(parents=True, exist_ok=True)

zip_paths = [Path("/content/phonation.zip"), Path("/content/prosody.zip")]  # adjust if names differ

for zp in zip_paths:
    if not zp.exists():
        print("Missing:", zp)
        continue
    with zipfile.ZipFile(zp, "r") as z:
        z.extractall(extract_root)

print("Extracted to:", extract_root)


Extracted to: /content/all_feature_csvs


In [ ]:
# ========= B2) Find all CSVs =========
csv_paths = sorted(extract_root.rglob("*.csv"))
print("CSV files found:", len(csv_paths))
print("First 10:", [p.name for p in csv_paths[:10]])


CSV files found: 15
First 10: ['cpp.csv', 'd2_vowel_A.csv', 'd2_vowel_E.csv', 'd2_vowel_I.csv', 'd2_vowel_O.csv', 'd2_vowel_U.csv', 'hnr_stats_50_4000.csv', 'jitter_by_vowel.csv', 'ppe.csv', 'shimmer_by_vowel.csv']


In [ ]:
# ========= B3) Read each CSV -> create person_id from last 4 digits -> aggregate per person =========
import pandas as pd
import numpy as np
import re

def pick_file_col(df):
    # common column names used in your notebooks
    for c in ["person_id", "speaker_id", "subject"]:
        if c in df.columns:
            return c  # already an id col
    for c in ["file", "filename", "wav", "wav_path", "path"]:
        if c in df.columns:
            return c
    return None

def add_person_id(df):
    df = df.copy()
    id_col = pick_file_col(df)

    if id_col is None:
        df["person_id"] = np.nan
        return df

    if id_col in ["person_id", "speaker_id", "subject"]:
        df["person_id"] = df[id_col].astype(str)
        return df

    # else derive from last 4 digits in filename/path
    s = df[id_col].astype(str)
    df["person_id"] = s.str.extract(r"(\d{4})(?!.*\d)", expand=False)
    # fallback: if extraction fails, keep the whole string (won't merge, but won't crash)
    df["person_id"] = df["person_id"].fillna(s)
    return df

def aggregate_per_person(df):
    df = add_person_id(df)
    df = df[df["person_id"].notna()]

    # keep label if exists (mode)
    label_series = None
    if "label" in df.columns:
        label_series = (df.groupby("person_id")["label"]
                        .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0]))

    # numeric features: aggregate by mean
    num_cols = [c for c in df.columns if c not in ["person_id", "label"] and pd.api.types.is_numeric_dtype(df[c])]
    agg = df.groupby("person_id")[num_cols].mean() if num_cols else pd.DataFrame(index=df["person_id"].unique())

    if label_series is not None:
        agg = agg.join(label_series)

    agg = agg.reset_index()
    return agg

tables = []
names = []

for p in csv_paths:
    try:
        df = pd.read_csv(p)
        if len(df) == 0:
            continue
        agg = aggregate_per_person(df)
        if len(agg) == 0:
            continue

        # prefix feature columns with the file stem to avoid name collisions
        prefix = re.sub(r"[^A-Za-z0-9]+", "_", p.stem).strip("_")
        keep = ["person_id"]
        for c in agg.columns:
            if c == "person_id":
                continue
            if c == "label":
                keep.append("label")  # keep label unprefixed
            else:
                agg.rename(columns={c: f"{prefix}__{c}"}, inplace=True)
                keep.append(f"{prefix}__{c}")

        agg = agg[keep]
        tables.append(agg)
        names.append(p.name)

    except Exception as e:
        print("Failed reading:", p.name, "|", e)

print("Usable CSVs:", len(tables))
print("Example sources:", names[:10])


Usable CSVs: 15
Example sources: ['cpp.csv', 'd2_vowel_A.csv', 'd2_vowel_E.csv', 'd2_vowel_I.csv', 'd2_vowel_O.csv', 'd2_vowel_U.csv', 'hnr_stats_50_4000.csv', 'jitter_by_vowel.csv', 'ppe.csv', 'shimmer_by_vowel.csv']


In [ ]:
# ========= B4) Outer-merge all tables on person_id and save =========
from functools import reduce

def merge_two(a, b):
    # if both have label, keep one (prefer a)
    if "label" in a.columns and "label" in b.columns:
        b = b.drop(columns=["label"])
    return a.merge(b, on="person_id", how="outer")

merged = reduce(merge_two, tables)

# If label exists, move it next to person_id
cols = ["person_id"] + (["label"] if "label" in merged.columns else []) + \
       [c for c in merged.columns if c not in ["person_id", "label"]]
merged = merged[cols]

out_path = "/content/all_features_merged_by_person.csv"
merged.to_csv(out_path, index=False)
print("Saved:", out_path)
print("Rows (persons):", len(merged), "| Columns:", merged.shape[1])

merged.head()


Saved: /content/all_features_merged_by_person.csv
Rows (persons): 215 | Columns: 37


,person_id,label,cpp__cpp_mean,cpp__cpp_median,cpp__cpp_std,d2_vowel_A__d2,d2_vowel_E__d2,d2_vowel_I__d2,d2_vowel_O__d2,d2_vowel_U__d2,...,f0_variability_espontanea__voiced_ratio,pause_features_espontanea__pause_count,pause_features_espontanea__pause_mean_sec,pause_features_espontanea__pause_total_sec,pause_features_espontanea__pause_ratio,speech_rate_proxy_espontanea__duration_sec,speech_rate_proxy_espontanea__speech_time_sec,speech_rate_proxy_espontanea__articulation_rate,speech_rate_proxy_espontanea__speech_segment_rate,speech_rate_proxy_espontanea__mean_speech_segment_sec
0,0004,PD,0.353981,0.344951,0.099406,0.129166,2.718906,2.952226,2.246848,2.608623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0006,PD,0.868925,0.870128,0.130662,2.250502,2.681773,3.007183,2.865828,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0007,PD,0.965410,0.938509,0.178515,2.085396,2.156636,2.640172,2.065918,2.420016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0008,PD,0.829396,0.831952,0.123082,2.522824,2.446111,2.922354,2.716041,2.714279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0009,PD,0.748561,0.746542,0.143429,2.289623,2.893322,2.635358,2.257305,3.155156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
